# Reports
1. Inputs: analytics and metrics
2. Outputs: results and conclusions
3. ...

# 1.0 Business Problem

## 1.1 Contexto:
A Insurance All é uma empresa que fornece seguro de saúde para seus clientes e o time de produtos está analisando a possibilidade de oferecer aos assegurados, um novo produto: Um seguro de automóveis.

Assim como o seguro de saúde, os clientes desse novo plano de seguro de automóveis precisam pagar um valor anualmente à Insurance All para obter um valor assegurado pela empresa, destinado aos custos de um eventual acidente ou dano ao veículo.

A Insurance All fez uma pesquisa com cerca de 380 mil clientes sobre o interesse em aderir a um novo produto de seguro de automóveis, no ano passado. Todos os clientes demonstraram interesse ou não em adquirir o seguro de automóvel e essas respostas ficaram salvas em um banco de dados junto com outros atributos dos clientes.

O time de produtos selecionou 127 mil novos clientes que não responderam a pesquisa para participar de uma campanha, no qual receberão a oferta do novo produto de seguro de automóveis. A oferta será feita pelo time de vendas através de ligações telefônicas.

Projeto retirado do Blog Seja Um Data Scientist
https://sejaumdatascientist.com/como-usar-data-science-para-fazer-a-empresa-vender-mais/

Projeto original disponível na plataforma Kaggle
https://www.kaggle.com/anmolkumar/health-insurance-cross-sell-prediction


## 1.2 Estratégia da Solução:

###### 1.2.1 Objetivos:
1. Principais Insights sobre os atributos mais relevantes de clientes interessados em adquirir um seguro de automóvel.
2. Qual a porcentagem de clientes interessados em adquirir um seguro de automóvel, o time de vendas conseguirá contatar fazendo 20.000 ligações?
3. E se a capacidade do time de vendas aumentar para 40.000 ligações, qual a porcentagem de clientes interessados em adquirir um seguro de automóvel o time de vendas conseguirá contatar?
4. Quantas ligações o time de vendas precisa fazer para contatar 80% dos clientes interessados em adquirir um seguro de automóvel?

###### 1.2.2 Proposta de Solução:
1. Gráficos e estudos de como as features se relacionam com a variávei alvo, individualmente ou combinadas através de Exploratory Data Analysis e Descriptive Statistics.
2. Treinar um modelo de Machine Learning, scorar os clientes, priorizar os clientes segundo suas probabilidades de sucesso, utilizar das métricas para criar cenários com os 20.000 melhores.
3. Utilizar do modelo anterior e criar cenários com os 40.000 melhores.
4. Utilizar do modelo anterior, definir curva de probabilidades para todos os clientes e identificar ponto crítico.
5. Fazer deploy e disponibilizar online através do Heroku

###### 1.2.3 Inputs:
1. Problema de negócio
2. Conjunto de dados com características de 380.000 clientes e as suas respostas sobre interesse em adquirir um seguro de automóveis.

###### 1.2.4 Outputs:
1. Gráficos e parâmetros em uma tabela no notebook (Gráfico / Parâmetros)
2. Resposta objetiva em um notebook (Uma porcentagem)
3. Resposta objetiva em um notebook (Uma porcentagem)
4. Resposta objetiva em um notebook (Um número absoluto)
5. Tabela de banco de dados SQL com todas as classificações
6. API com request/predict do tipo: | v1 | v2 | v3 | - 1/0

###### 1.2.5 Ciclos:
1. Pipeline de dados, funcional, de ponta a ponta (desde a coleta de dados até o 
    treinamento do modelo)
2. Entender os dados e limpa-los (buscar inconsistências) Análises de estatística
    descritiva de primeira ordem
3. Feature Engineering (criar variáveis que modelam o fenômeno)
4. Criar as hipóteses de negócio
5. Análise exploratória de dados para validar ou refutar hióteses
6. Definir métricas e treinar o modelo
7. Analisar métricas
8. Criar arquivo handle para todas as transformações necessárias
9. Criar API
10. Resultados e Conclusão

## 1.3 Sumário:

# 2.0 Imports

###### 2.1 Libraries

In [1]:
import numpy as np
import pandas as pd
import pickle

from sklearn.metrics import classification_report, roc_auc_score, accuracy_score

import time
import warnings

# Tempo de processamento
time_agora = time.time()

###### 2.2 Loading dataset

In [2]:
# raw data
train = pd.read_csv('D:\\My Drive\\Pessoal\\Projetos\\insurance_sales_predict\\insurance_sales_predict\\train.csv')
test = pd.read_csv('D:\\My Drive\\Pessoal\\Projetos\\insurance_sales_predict\\insurance_sales_predict\\test.csv')
df = pd.concat([train, test])

# arrays
X = pickle.load(open('D:\\My Drive\\Pessoal\\Projetos\\insurance_sales_predict\\insurance_sales_predict\\exportings\\X_to_modeling.pkl', 'rb'))
y = pickle.load(open('D:\\My Drive\\Pessoal\\Projetos\\insurance_sales_predict\\insurance_sales_predict\\exportings\\y_to_modeling.pkl', 'rb'))
X_train = X[2]
y_train = y[6]

# processed data
df_predict_request = pickle.load(open('D:\\My Drive\\Pessoal\\Projetos\\insurance_sales_predict\\insurance_sales_predict\\exportings\\df_predict_request.pkl', 'rb'))
df_predict_project = pickle.load(open('D:\\My Drive\\Pessoal\\Projetos\\insurance_sales_predict\\insurance_sales_predict\\exportings\\df_predict_project.pkl', 'rb'))

# columns
model_columns = pickle.load(open('D:\\My Drive\\Pessoal\\Projetos\\insurance_sales_predict\\insurance_sales_predict\\exportings\\model_columns.pkl', 'rb'))

# model
lr_model = pickle.load(open('D:\\My Drive\\Pessoal\\Projetos\\insurance_sales_predict\\deploy\\model\\lr_model.pkl', 'rb'))

# 3.0 Baseline

In [3]:
baseline = train['Response'].mean()
print(f'A média atual de interesse no produto é igual a {round(baseline, 2)*100}%')

A média atual de interesse no produto é igual a 12.0%


# 4.0 Métricas

Modelo XGRFBoost: PRECISION de 0.27, RECALL de 0.94 e ROC_AUC 0.80

# 5.0 Principais Insights sobre os atributos mais relevantes de clientes interessados em adquirir um seguro de automóvel.

### Espaço Amostral
1. 6 dimensões parecem explicar 90% de toda variabilidade das features
2. Annual_Premium transformada por log apresenta uma shape semelhante a uma normal
3. 4 features binárias, 2 numéricas, 1 categórica e 2 categóricas com excessivas categorias e portanto foram transformadas em numéricas.

### Features Analysis

###### Response:
1. Desbalanceada com 12% de eventos positivos
2. Correlação fraca com Vehicle_Damage e correlação fraca negativa com Previously_Insured

###### Gender:
1. Balanceada
2. Correlações muito fracas
3. Homens se interessam 30% mais no produto em relação às mulheres

###### Vehicle_Age:
1. Mais da metade dos clientes possuem carros com menos de 1 ano
2. Correlação moderada negativa com Age, fraca positiva com Policy_Sales_Channel
3. Médias de response bastante distintas

###### Vehicle_Damage:
1. Balanceada
2. Correlação muito forte negativa com Previously_Insured, fraca negativa com Policy_Sales_Channel, fraca positiva com Response e Age
3. Clientes que já tiveram danos no seu veículo tem 24% de response, enquanto clientes que não tiveram danos têm por volta de 1%

###### Driving_License:
1. Correlações muito fracas
2. 99% dos clientes são habilitados
3. Habilitados apresentam 5% de interesse no produto, não habilitados apresentam 12% de interesse no produto.

###### Previously_Insured:
1. 45% dos clientes já possuem seguro veicular
2. Clientes que já possuem seguro têm interesse quase zero pelo produto
3. Correlação muito forte negativa com Vehicle_Damage, fraca negativa com Response e Age, e fraca positiva com Policy_Sales_Channel

###### Region_Code:
1. 53 regiões
2. 28% dos clientes está contido na região 28. 50% dos dados estão contidos nas 5 regiões mais numerosas
3. A região 28 (mais importante) tem por volta de 18% de probabilidade de response, segunda melhor região, atrás apenas da região 38.
4. As regiões com probabilidade de response mais alto estão com por volta de 19%, as regiões com probabilidades mais baixas estã em por volta de 4%
5. Médias de response bastante distintas, com desvios padrões baixos

###### Policy_Sales_Channel:
1. Há 157 Policy_Sales_Channel e aproximadamente um terço das amostras está contido no canal 152
2. 5 categorias contém 84.19% do total de clientes
3. Correlação moderada negativa com Age, correlação fraca negativa com Vehicle_Damage, correlação fraca positiva com Vehicle_Age e Previously_Insured
4. Médias de response por canais de Policy_Sales_Channel bastante distintas
5. Os canais mais utilizados apresentam médias de response bastante diferentes da média global, os canal 152 é o mais utilizado e apresenta 2% de interesse no produto, o canal 155 apresenta 2%, já os canais 26, 124 e 156 apresentam por volta de 20% de interesse no produto

###### Vintage:
1. Feature categórica, há 290 categorias em Vintage

###### Age:
1. Range de idade de 25 até 85 anos, aparentemente sem erros de input
2. Apresenta uma distribuição com uma certa semelhança com uma Poison, mas com uma segunda moda em um ponto próximo aos 45 anos.
3. Sem outliers
4. A média de idade entre os que não se interessam pelo produto é de 38 anos, e dos que se interessam é de 43 anos.
5. Pessoas com mais de 30 anos apresentam um interesse relativamente maior pelo produto.
6. Correlações moderadas negativas com Policy_Sales_Channel e Vehicle_Age, fraca negativa com Previously_Insured e fraca positiva com Vehicle_Damage
7. Clientes com menos de 30 anos de idade apresentam em média 4% de interesse no produto, enquanto os clientes com 30 anos ou mais têm em média 17% interesse no produto.


###### Annual_Premium
1. 2799 em Annual_Premmium, não aparentam ser por conta de erros de input
2. Há dois grupos de clientes, um com valores fixos em 2630 dollares, e outro com valores contínuos e superiores, não há diferença significativa de média de interesse no produto entre os grupos
3. Acima do terceiro quartil da distribuição há um interesse relativo 20% maior no produto

# 6.0 Qual a porcentagem de clientes interessados em adquirir um seguro de automóvel, o time de vendas conseguirá contatar fazendo 20.000 ligações?

### 6.1 Premissas

###### 6.1.1 Through request

In [4]:
df_predict_request.shape

(381109, 13)

In [5]:
df_predict_request.sort_index(inplace=True)

In [6]:
request_20000 = df_predict_request.sort_values('probability', ascending=False).head(20000)

In [7]:
request_20000.head(5)

,Age,Previously_Insured,Gender_,Vehicle_Damage_,Policy_Sales_Channel_Response_Mean,Region_Code_Response_Mean,High_Response_Age,Log_Annual_Premium,Vehicle_Age_1,Vehicle_Age_2,Response,predictions,probability
228805,0.398653,-0.920227,0.921495,0.991555,10.159910,1.426486,0.827798,0.944213,-0.871583,-0.210045,1,1.0,0.996612
24016,0.398653,-0.920227,-1.085193,0.991555,10.159910,1.426486,0.827798,0.747009,-0.871583,-0.210045,1,1.0,0.996215
116011,-0.246481,-0.920227,0.921495,0.991555,2.286915,1.426486,0.827798,0.767959,-0.871583,4.760882,0,1.0,0.929209
241121,-0.181968,-0.920227,0.921495,0.991555,2.286915,1.426486,0.827798,0.636769,-0.871583,4.760882,0,1.0,0.926569
12738,-0.375508,-0.920227,-1.085193,0.991555,2.286915,1.426486,0.827798,0.682032,-0.871583,4.760882,0,1.0,0.926526


In [8]:
print(classification_report(request_20000['Response'], request_20000['predictions'], zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     12399
           1       0.38      1.00      0.55      7601

    accuracy                           0.38     20000
   macro avg       0.19      0.50      0.28     20000
weighted avg       0.14      0.38      0.21     20000



###### 6.1.2 Through projetct

In [9]:
pred = lr_model.predict(X_train)

In [10]:
proba = lr_model.predict_proba(X_train)[:,1]

In [11]:
df_predict_project = pd.DataFrame(X_train, columns=model_columns)

In [12]:
df_predict_project['predictions'] = pred
df_predict_project['probability'] = proba
df_predict_project['Response'] = y_train

In [13]:
df_predict_project.sort_index(inplace=True)

In [14]:
project_20000 = df_predict_project.sort_values('probability', ascending=False).head(20000)

In [15]:
project_20000.head(5)

,Age,Previously_Insured,Gender_,Vehicle_Damage_,Policy_Sales_Channel_Response_Mean,Region_Code_Response_Mean,High_Response_Age,Log_Annual_Premium,Vehicle_Age_1,Vehicle_Age_2,predictions,probability,Response
228805,0.398653,-0.920227,0.921495,0.991555,10.159910,1.426486,0.827798,0.944213,-0.871583,-0.210045,1.0,0.996533,1.0
24016,0.398653,-0.920227,-1.085193,0.991555,10.159910,1.426486,0.827798,0.747009,-0.871583,-0.210045,1.0,0.996138,1.0
116011,-0.246481,-0.920227,0.921495,0.991555,2.286915,1.426486,0.827798,0.767959,-0.871583,4.760882,1.0,0.928266,0.0
12738,-0.375508,-0.920227,-1.085193,0.991555,2.286915,1.426486,0.827798,0.682032,-0.871583,4.760882,1.0,0.925695,0.0
241121,-0.181968,-0.920227,0.921495,0.991555,2.286915,1.426486,0.827798,0.636769,-0.871583,4.760882,1.0,0.925627,0.0


In [16]:
print(classification_report(project_20000['Response'], project_20000['predictions'], zero_division=0))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00     12412
         1.0       0.38      1.00      0.55      7588

    accuracy                           0.38     20000
   macro avg       0.19      0.50      0.28     20000
weighted avg       0.14      0.38      0.21     20000



###### 6.1.3 Testendo processamento do handler x projeto

In [17]:
np.sum(np.abs(df_predict_project - df_predict_request))

Age                                     0.000011
Gender_                                 0.000013
High_Response_Age                       0.000009
Log_Annual_Premium                      0.000009
Policy_Sales_Channel_Response_Mean      0.000007
Previously_Insured                      0.000007
Region_Code_Response_Mean               0.000006
Response                                0.000000
Vehicle_Age_1                           0.000009
Vehicle_Age_2                           0.000015
Vehicle_Damage_                         0.000009
predictions                           339.000000
probability                           208.479702
dtype: float64

### 6.2 Calculos

# 7.0 E se a capacidade do time de vendas aumentar para 40.000 ligações, qual a porcentagem de clientes interessados em adquirir um seguro de automóvel o time de vendas conseguirá contatar?

### 7.1 Premissas

### 7.2 Calculos

# 8.0 Quantas ligações o time de vendas precisa fazer para contatar 80% dos clientes interessados em adquirir um seguro de automóvel?

### 8.1 Premissas

### 8.2 Calculos

# Time

In [18]:
print(f'O tempo de processamento do projeto foi de: {int(round(time.time()-time_agora, 2)/60)} minutos')

O tempo de processamento do projeto foi de: 0 minutos
